In [ ]:
ma_context_diffimport os

import pandas as pd

""" Loads data saved in /rsa
- TO MAKE PLOTS, which combined all the above, go to 231203_snippets_rsa_PLOTS.ipynb

"""

In [ ]:
%load_ext autoreload
%autoreload 2

from neuralmonkey.classes.session import load_mult_session_helper
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline

### MULT RES PLOTS

In [ ]:
import pickle
import seaborn as sns
import pandas as pd
from pythonlib.tools.pandastools import grouping_print_n_samples, append_col_with_grp_index
from pythonlib.tools.plottools import rotate_x_labels, rotate_y_labels, savefig
from pythonlib.tools.snstools import rotateLabel
from pythonlib.tools.expttools import writeDictToYaml


In [ ]:
# # from neuralmonkey.analyses.state_space_good import pipeline_rsa_scalar_population_MULT, load_mult_data_helper, pipeline_rsa_scalar_population_MULT_PLOTS, load_single_data
# from neuralmonkey.analyses.rsa import pipeline_rsa_scalar_population_MULT, load_mult_data_helper, pipeline_rsa_scalar_population_MULT_PLOTS, _load_single_data


In [ ]:
animal = "Diego"
DATE = 230630
version_distance = "pearson"
RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, version_distance)

In [ ]:
# Load a specific bregion and time window

res, PA, Clraw, Clsim = load_single_data(RES, "M1_m", (-0.5, -0.3), which_level="stroke")
from neuralmonkey.analyses.rsa import pipeline_rsa_scalar_population_MULT

## MAIN PLOTS

In [ ]:
if False:
    # PIG
    list_question = ["seq_ctxt"]
    # list_question = ["pig_vs_sp", "seq_ctxt", "seq_pred", "shape_loc"]
    # list_animal_date = [
    #         ("Diego", "230628"),
    #         ("Diego", "230630"),
    #         ("Pancho", "230623"),
    #         ("Pancho", "230626"),
    #         ]
    list_animal_date = [
            ("Pancho", "230623"),
            # ("Pancho", "230623"),
            # ("Pancho", "230626"),
            ]
    list_which_level = ["stroke", "stroke_off"]
elif False:
    # Char
    list_question = ["CHAR_shape"]
    list_animal_date = [
            ("Pancho", "230126"),
            ]
    list_which_level = ["stroke", "stroke_off"]
elif False:
    list_question = ["SP_shape_size"]
    list_animal_date = [
            ("Diego", "230615"),
        ]
    animal = "Diego"
    # list_question = ["SP_shape"]    
    # list_animal_date = [
    #         ("Diego", "230615"),
    #         ]
    
    list_which_level = ["trial"]
else:    
    list_question = ["SP_shape_loc"]
    list_animal_date = [
            ("Diego", "230615"),
        ]
    list_which_level = ["trial"]



list_version_distance = ["pearson"]
list_yvar = ["cc"]
DO_AGG_TRIALS = True

In [ ]:
from neuralmonkey.analyses.rsa import rsagood_pa_vs_theor_wrapper_loadresults, rsagood_pa_vs_theor_plot_results
subtract_mean_each_level_of_var = None
yvar = "cc"
for question in list_question:    
    for animal, date in list_animal_date:
        for version_distance in list_version_distance:
            for yvar in list_yvar:
                DFRES_THEOR, DFallpa, Params, savedir = rsagood_pa_vs_theor_wrapper_loadresults(animal, date, question, version_distance, DO_AGG_TRIALS, subtract_mean_each_level_of_var)
                    
                ### (1) Overview summary plots
                print("** SAving figures to:", savedir)
                rsagood_pa_vs_theor_plot_results(DFRES_THEOR, savedir, yvar)
                
                ### (2) PLOT PAIRWISE DIST MATRICES, for hand-selected variables.
                # Reload the params for this question.
                from neuralmonkey.analyses.rsa import rsagood_questions_params
                question_params = rsagood_questions_params(Params["question"])
                variables_plot = question_params["plot_pairwise_distmats_variables"]
                list_wl_ev_tw_plot = question_params["plot_pairwise_distmats_twinds"]
                
                rsagood_pa_vs_theor_plot_pairwise_distmats(DFallpa, Params, savedir, variables_plot,
                                                           list_wl_ev_tw_plot, DO_AGG_TRIALS_PLOT=True)
    
                

### Sumamrizing all results across analyses (combining across questions, taking each var only one time)

In [ ]:
# Which vars to extract for each question, for global analysis.
import os
map_questions_to_vars_take = {
    "shape_loc":["shape_oriented", "gridloc", "gap_from_prev_angle_binned"],
    "pig_vs_sp":["task_kind"],
    "seq_ctxt":["stroke_index_fromlast_tskstks"],
    "seq_pred":["CTXT_loc_next", "CTXT_shape_next", "gap_to_next_angle_binned"],
}

map_questions_to_vars_take = {
    "SP_shape_size":["seqc_0_shape", "gridsize"]
}

for animal, DATE in list_animal_date:
    for version_distance in list_version_distance:
        for yvar in list_yvar:
            
            # "sanity that get each var 1x"
            allvars = []
            for vars in map_questions_to_vars_take.values():
                allvars.extend(vars)
            assert len(set(allvars))==len(allvars)
            
            list_df = []
            for question in list_question:
                
                ########## LOAD ALL DATA
                DFRES_THEOR, DFallpa, Params, savedir = rsagood_pa_vs_theor_wrapper_loadresults(animal, date, question, version_distance, DO_AGG_TRIALS, subtract_mean_each_level_of_var)
                        
                DFRES_THEOR["version_distance"] = version_distance
                DFRES_THEOR["question"] = question
                vars_keep = map_questions_to_vars_take[question]
                DFRES_THEOR = DFRES_THEOR[DFRES_THEOR["var"].isin(vars_keep)]
                list_df.append(DFRES_THEOR)    

                # ## COLLECT
                # # list_which_level = []
                # for res in RES:
                #     which_level = res["which_level"]
                #     version_distance = res["version_distance"]
                #     df = res["DFRES_THEOR"]
                #     df["which_level"] = which_level
                #     df["version_distance"] = version_distance
                #     df["question"] = question
                #     
                #     # Only pull out specific vars, depending on the question
                #     
                #     vars_keep = map_questions_to_vars_take[question]
                #     dfkeep = df[df["var"].isin(vars_keep)]
                #     list_df.append(dfkeep)    
                # 
            # list_which_level = sorted(set(list_which_level))
            DFMULT_THEOR = pd.concat(list_df).reset_index(drop=True)
            DFMULT_THEOR = append_col_with_grp_index(DFMULT_THEOR, ["question", "var"], "q-var", strings_compact=True)
            # DFMULT_THEOR["var"] = DFMULT_THEOR["q-var"]
            
            # CHeck that you got all vars
            for var in allvars:
                print(var)
                assert var in DFMULT_THEOR["var"].unique()
            
            from neuralmonkey.analyses.rsa import SAVEDIR_ANALYSES
            savedir = f"{SAVEDIR_ANALYSES}/{animal}-{date}/agg_{DO_AGG_TRIALS}-subtr_{subtract_mean_each_level_of_var}-dist_{version_distance}/COMBINED_ACROSS_QUESTIONS_FINAL"
            print(savedir)
            os.makedirs(savedir, exist_ok=True)
            
            DFMULT_THEOR, DictBregionToDf2d, DictVarToDf2d, dfres_kernels_2d, PARAMS = pipeline_rsa_scalar_population_MULT_PLOTS(DFMULT_THEOR, savedir, yvar)
        

### Deeper dive into variable conjucntions and imbalances. [Separately for each question]

In [ ]:
from neuralmonkey.analyses.rsa import pipeline_rsa_scalar_population_MULT_PLOT_DETAILED 

In [ ]:
assert False, "NOT READY -- should intergrate this into original steps that do computation, rsagood_pa_vs_theor_wrapper.."
# savedir = f"{SAVEDIR_MULT}/effects_each_var"
# os.makedirs(savedir, exist_ok=True)
# print(savedir)

for question in list_question:    
    for animal, DATE in list_animal_date:
        for version_distance in list_version_distance:
            pipeline_rsa_scalar_population_MULT_PLOT_DETAILED(animal, DATE, version_distance, list_which_level, question, DO_AGG_TRIALS=DO_AGG_TRIALS, event=None)

In [ ]:
pipeline_rsa_scalar_population_MULT_PLOT_DETAILED

### Sumamrizing all results across analyses (combining across questions) [conjunction of question-var]

In [ ]:
assert False, "NOT USEFUL"
# Load all RES, then concatenate them so each variable is actually "question-varible"
for animal, DATE in list_animal_date:
    for version_distance in list_version_distance:
        for yvar in list_yvar:

            # COLLECT ACROSS QUESTIONS
            list_df = []
            for question in list_question:
                
                ########## LOAD ALL DATA
                RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, version_distance,
                                                                                    list_which_level=list_which_level,
                                                                                    question=question, DO_AGG_TRIALS=DO_AGG_TRIALS)
                
                ## COLLECT
                # list_which_level = []
                for res in RES:
                    which_level = res["which_level"]
                    version_distance = res["version_distance"]
                    df = res["DFRES_THEOR"]
                    df["which_level"] = which_level
                    df["version_distance"] = version_distance
                    df["question"] = question
                    list_df.append(df)
                #     list_which_level.append(which_level)
                
                
            # list_which_level = sorted(set(list_which_level))
            DFMULT_THEOR = pd.concat(list_df).reset_index(drop=True)
            DFMULT_THEOR = append_col_with_grp_index(DFMULT_THEOR, ["question", "var"], "q-var", strings_compact=True)
            DFMULT_THEOR["var"] = DFMULT_THEOR["q-var"]
            
            savedir = f"{params['SAVEDIR']}/{animal}/SPLIT_BY_QUESTIONS/MULT/{DATE}/{version_distance}/COMBINED_ACROSS_QUESTIONS"
            print(savedir)
            os.makedirs(savedir, exist_ok=True)
            
            pipeline_rsa_scalar_population_MULT_PLOTS(DFMULT_THEOR, savedir, yvar)

#### Compute decodability of the variable

## OLD (all moved above!!)

In [ ]:
### Sumamrizing all results across analyses (NOT QUESTIONS-VERSION)
from neuralmonkey.analyses.state_space_good import pipeline_rsa_scalar_population_MULT

list_animal_date = [
        ("Diego", "230628"),
        ("Diego", "230630"),
        ("Pancho", "230623"),
        ("Pancho", "230626"),
        ]
# list_version_distance = ["pearson", "euclidian"]
# list_yvar = ["cc", "mr_coeff"]
list_version_distance = ["pearson"]
list_yvar = ["cc"]

for animal, DATE in list_animal_date:
    for version_distance in list_version_distance:
        for yvar in list_yvar:
            pipeline_rsa_scalar_population_MULT(animal, DATE, version_distance, yvar)
            assert False
### Sumamrizing all results, indivudually for each analyses (i.e., question)

from neuralmonkey.analyses.rsa import pipeline_rsa_scalar_population_MULT
if False:
    # PIG
    list_question = ["seq_ctxt"]
    # list_question = ["pig_vs_sp", "seq_ctxt", "seq_pred", "shape_loc"]
    # list_animal_date = [
    #         ("Diego", "230628"),
    #         ("Diego", "230630"),
    #         ("Pancho", "230623"),
    #         ("Pancho", "230626"),
    #         ]
    list_animal_date = [
            ("Pancho", "230623"),
            # ("Pancho", "230623"),
            # ("Pancho", "230626"),
            ]
    list_which_level = ["stroke", "stroke_off"]
elif False:
    # Char
    list_question = ["CHAR_shape"]
    list_animal_date = [
            ("Pancho", "230126"),
            ]
    list_which_level = ["stroke", "stroke_off"]
elif False:
    list_question = ["SP_shape_size"]
    list_animal_date = [
            ("Diego", "230615"),
        ]
    animal = "Diego"
    # list_question = ["SP_shape"]    
    # list_animal_date = [
    #         ("Diego", "230615"),
    #         ]
    
    list_which_level = ["trial"]
else:    
    list_question = ["SP_shape_size"]
    list_animal_date = [
            ("Pancho", "220606"),
        ]
    animal = "Pancho"    
    list_which_level = ["trial"]



list_version_distance = ["pearson"]
list_yvar = ["cc"]
DO_AGG_TRIALS = True
for question in list_question:    
    for animal, DATE in list_animal_date:
        for version_distance in list_version_distance:
            for yvar in list_yvar:
                pipeline_rsa_scalar_population_MULT(animal, DATE, version_distance, yvar, list_which_level=list_which_level, question=question, DO_AGG_TRIALS=DO_AGG_TRIALS, event=None)
### Deeper dive into variable conjucntions and imbalances. [Separately for each question]
from neuralmonkey.analyses.rsa import pipeline_rsa_scalar_population_MULT_PLOT_DETAILED 
# savedir = f"{SAVEDIR_MULT}/effects_each_var"
# os.makedirs(savedir, exist_ok=True)
# print(savedir)

for question in list_question:    
    for animal, DATE in list_animal_date:
        for version_distance in list_version_distance:
            pipeline_rsa_scalar_population_MULT_PLOT_DETAILED(animal, DATE, version_distance, list_which_level, question, DO_AGG_TRIALS=DO_AGG_TRIALS, event=None)
### Sumamrizing all results across analyses (combining across questions) [conjunction of question-var]
# Load all RES, then concatenate them so each variable is actually "question-varible"
for animal, DATE in list_animal_date:
    for version_distance in list_version_distance:
        for yvar in list_yvar:

            # COLLECT ACROSS QUESTIONS
            list_df = []
            for question in list_question:
                
                ########## LOAD ALL DATA
                RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, version_distance,
                                                                                    list_which_level=list_which_level,
                                                                                    question=question, DO_AGG_TRIALS=DO_AGG_TRIALS)
                
                ## COLLECT
                # list_which_level = []
                for res in RES:
                    which_level = res["which_level"]
                    version_distance = res["version_distance"]
                    df = res["DFRES_THEOR"]
                    df["which_level"] = which_level
                    df["version_distance"] = version_distance
                    df["question"] = question
                    list_df.append(df)
                #     list_which_level.append(which_level)
                
                
            # list_which_level = sorted(set(list_which_level))
            DFMULT_THEOR = pd.concat(list_df).reset_index(drop=True)
            DFMULT_THEOR = append_col_with_grp_index(DFMULT_THEOR, ["question", "var"], "q-var", strings_compact=True)
            DFMULT_THEOR["var"] = DFMULT_THEOR["q-var"]
            
            savedir = f"{params['SAVEDIR']}/{animal}/SPLIT_BY_QUESTIONS/MULT/{DATE}/{version_distance}/COMBINED_ACROSS_QUESTIONS"
            print(savedir)
            os.makedirs(savedir, exist_ok=True)
            
            pipeline_rsa_scalar_population_MULT_PLOTS(DFMULT_THEOR, savedir, yvar)
### Sumamrizing all results across analyses (combining across questions, taking each var only one time)
# Which vars to extract for each question, for global analysis.
import os
map_questions_to_vars_take = {
    "shape_loc":["shape_oriented", "gridloc", "gap_from_prev_angle_binned"],
    "pig_vs_sp":["task_kind"],
    "seq_ctxt":["stroke_index_fromlast_tskstks"],
    "seq_pred":["CTXT_loc_next", "CTXT_shape_next", "gap_to_next_angle_binned"],
}


for animal, DATE in list_animal_date:
    for version_distance in list_version_distance:
        for yvar in list_yvar:
            
            # "sanity that get each var 1x"
            allvars = []
            for vars in map_questions_to_vars_take.values():
                allvars.extend(vars)
            assert len(set(allvars))==len(allvars)
            
            list_df = []
            for question in list_question:
                
                ########## LOAD ALL DATA
                RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, version_distance,
                                                                                    list_which_level=list_which_level,
                                                                                    question=question, DO_AGG_TRIALS=DO_AGG_TRIALS)
                
                ## COLLECT
                # list_which_level = []
                for res in RES:
                    which_level = res["which_level"]
                    version_distance = res["version_distance"]
                    df = res["DFRES_THEOR"]
                    df["which_level"] = which_level
                    df["version_distance"] = version_distance
                    df["question"] = question
                    
                    # Only pull out specific vars, depending on the question
                    
                    vars_keep = map_questions_to_vars_take[question]
                    dfkeep = df[df["var"].isin(vars_keep)]
                    list_df.append(dfkeep)    
                
            # list_which_level = sorted(set(list_which_level))
            DFMULT_THEOR = pd.concat(list_df).reset_index(drop=True)
            DFMULT_THEOR = append_col_with_grp_index(DFMULT_THEOR, ["question", "var"], "q-var", strings_compact=True)
            # DFMULT_THEOR["var"] = DFMULT_THEOR["q-var"]
            
            # CHeck that you got all vars
            for var in allvars:
                print(var)
                assert var in DFMULT_THEOR["var"].unique()
            
            savedir = f"{params['SAVEDIR']}/{animal}/SPLIT_BY_QUESTIONS/MULT/{DATE}/{version_distance}/COMBINED_ACROSS_QUESTIONS_FINAL"
            print(savedir)
            os.makedirs(savedir, exist_ok=True)
            
            DFMULT_THEOR, DictBregionToDf2d, DictVarToDf2d, dfres_kernels_2d, PARAMS = pipeline_rsa_scalar_population_MULT_PLOTS(DFMULT_THEOR, savedir, yvar)
        
### Plotting dist matrices, sorted in various ways
from neuralmonkey.analyses.rsa import _pipeline_rsa_score_pa_single
import os

# twind_keep = (0.1, 0.3)
# twind_keep = (0.3, 0.5)
twind_keep = (-0.3, -0.1)

question = "SP_shape"
variable_pair = ["seqc_0_shape", "seqc_0_loc"]

# question = "SP_shape_size"
# variable_pair = ["seqc_0_shape", "gridsize"]

DO_AGG_TRIALS_LOAD = True
DO_AGG_TRIALS_PLOT = True
version_distance = "pearson"
list_which_level = ["trial"]


# 
# # twind_keep = (0.1, 0.3)
# twind_keep = (-0.1, 0.1)
# # variable_pair = ["shape_label", "stroke_index"]
# variable_pair = ["shape_label", "velmean_thbin"]
# DO_AGG_TRIALS_LOAD = False
# DO_AGG_TRIALS_PLOT = True
# # animal = "Pancho"
# # DATE = 230623
# version_distance = "pearson"
# 
# # if False:
# # list_animal_date = [
# #     ("Pancho", "230623"),
# #     ("Pancho", "230626"),
# #     ("Diego", "230628"),
# #     ("Diego", "230630"),
# # ]
# question = "CHAR_shape"
for animal, DATE in list_animal_date:
    
    RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, version_distance, question=question, DO_AGG_TRIALS=DO_AGG_TRIALS_LOAD,
                                                                        list_which_level=list_which_level, event=None)
    savedir = f"{SAVEDIR_MULT}/replotting_pairwise_vars"
    os.makedirs(savedir, exist_ok=True)
    print(savedir)
    
    # resthis = []
    for res in RES:
        which_level=res["which_level"]
        for bregion in REGIONS_IN_ORDER:
            for twind in res["list_time_windows"]:
                if twind==twind_keep:
                    key = (bregion, twind)
                    print("Collecting data for:", res["which_level"], key)
                    PA = res["DictBregionTwindPA"][key]
                    EFFECT_VARS = res["EFFECT_VARS"]
            
                    # Norm of activity for each level of the variable. Then average these norms over the levels.    
                    for ivar, var1 in enumerate(EFFECT_VARS):
                        # condition this on each of the other vars
                        for jvar, var2 in enumerate(EFFECT_VARS):
                            if jvar>ivar:
                                if [var1, var2]==variable_pair:
                                    for subtract_mean_each_level_of_var in [None, var1, var2]:
                                        sdir = f"{savedir}/{which_level}-{bregion}{twind}/{var1}--{var2}--subtr_{subtract_mean_each_level_of_var}"
                                        os.makedirs(sdir, exist_ok=True)
                                        # Extract raw data
                                        
                                        _, _, Clraw, _, PAagg, DictVarToClsimtheor = _pipeline_rsa_score_pa_single(PA, [var1, var2], res["version_distance"], 
                                                                          subtract_mean_each_level_of_var, PLOT = True, sdir=sdir, PLOT_THEORETICAL_SIMMATS=True,
                                                                        COMPUTE_SAME_DIFF_DIST = False, COMPUTE_VS_THEOR_MAT = True, DO_AGG_TRIALS=DO_AGG_TRIALS_PLOT)
                                        
                                        plt.close("all") 
### Plotting rasters
##### Final summary plots (sandbox)
DictVarToDf2d["shape_oriented"]
dfres_kernels.keys()
# Hand-picked ... contrast the temporal structure of small set of variables
%matplotlib inline
# dfthis = DFMULT_THEOR
vars = ["gridloc", "gap_from_prev_angle_binned", "gap_to_next_angle_binned"]
dfthis = DFMULT_THEOR[DFMULT_THEOR["var"].isin(vars)]
sns.catplot(data=dfthis, x="twind_str", y=yvar, row="bregion", hue="var", col="which_level", kind="point")


# IN DEVELOPMENT

### Testing out, method for directly pitting two variables against each other

In [ ]:
# 1) Prune at level of comparisons: keep only comparisons that are opposed across the vars.
# -- What is the principle behind this? IT is more about the sign of the effect than the magnitude?
# -- More generally, is about getting subset of comparisons which are not correlated across the two vars.
# -- This seems weird?
# 2) Prune at level of var-levels: keep only levels (of both vars) that have at least 1 case of other var.
# -- This would get the "same beh" control in epochs.
# 3) "Kernels" using either the raw results, or the processed ones here (pairwise between variables).

#

In [ ]:
#TODO:
# - pick specific pair of variables. and plot across (bregion, timewindows). 
# - Make this saving pipleine

In [ ]:
# Pull out speciifc dataset (bregion x time)
which_level = "stroke"
version_distance = "pearson"
PLOT = False

savedir = f"{SAVEDIR}/{animal}/{DATE}/{which_level}/{version_distance}"
path = f"{savedir}/resthis.pkl"
print("Loading res from: ", path)
try:
    with open(path, "rb") as f:
        res = pickle.load(f)
except Exception as err:
    print(path)
    print("Couldnt load this data! *******************", version_distance, animal, DATE, which_level)

## EXTRACT
DictBregionTwindClraw = res["DictBregionTwindClraw"]
DictBregionTwindClsim = res["DictBregionTwindClsim"]
DictBregionTwindCPA = res["DictBregionTwindPA"]

res = []
for twind in list_twind:
    for bregion in list_bregion:
        # bregion = "PMd_p"
        key = (bregion, twind)
        Clraw = DictBregionTwindClraw[key]
        Clsim = DictBregionTwindClsim[key]
        PA = DictBregionTwindCPA[key]
        
        ##### 1. Given two variables, how similar are their theoretical sim mats?
        if PLOT:
            list_var1 =[]
            list_var2 = []
            list_cc = []
            for i, var1 in enumerate(EFFECT_VARS):
                for j, var2 in enumerate(EFFECT_VARS):
                    if j>i:
                        Cltheor1, _ = Clsim.rsa_distmat_construct_theoretical(var1, False)
                        Cltheor2, _ = Clsim.rsa_distmat_construct_theoretical(var2, False)
            
                        vec1 = Cltheor1.dataextract_upper_triangular_flattened()
                        vec2 = Cltheor2.dataextract_upper_triangular_flattened()
                        c = np.corrcoef(vec1, vec2)[0,1]
                        
                        print(var1, " -- ", var2, " -- ", c)
                        list_var1.append(var1)
                        list_var2.append(var2)
                        list_cc.append(c)
            
            dfthis = pd.DataFrame({"var1":list_var1, "var2":list_var2, "cc":list_cc})
            convert_to_2d_dataframe(dfthis, "var1", "var2", True, "mean", "cc", annotate_heatmap=True, dosort_colnames=False)
    
        # from neuralmonkey.analyses.state_space_good import rsa_convert_PA_to_Cl
        # rsa_convert_PA_to_Cl(DictBregionTwindPA.values[0])
        
        ##### Compare pair of vars
        # Go thru all pairs of vars
        PLOT = False
        for ivar, var in enumerate(EFFECT_VARS):
            for jvar, var_other in enumerate(EFFECT_VARS):
                if not var==var_other:
                # if jvar>ivar:
                    # var = "gridloc"
                    # var_other = "gap_from_prev_angle_binned"
                    # var = "shape_oriented"
                    # var_other = "gridloc"
                    
                    # find pairs of datapts which make different predictions across the two variables.
                    Cltheor1, _ = Clsim.rsa_distmat_construct_theoretical(var, False)
                    Cltheor2, _ = Clsim.rsa_distmat_construct_theoretical(var_other, False)
                    
                    # Extract actual data and theor data
                    vec_dat = Clsim.dataextract_upper_triangular_flattened()
                    vec1 = Cltheor1.dataextract_upper_triangular_flattened()
                    vec2 = Cltheor2.dataextract_upper_triangular_flattened()
                    
                    cc1 = np.corrcoef(vec_dat, vec1)[0,1]
                    cc2 = np.corrcoef(vec_dat, vec2)[0,1]
                    print(var, cc1)
                    print(var_other, cc2)
                    
                    cc_kind = "actual"
                    res.append({
                        "var":var,
                        "var_other":var_other,
                        "cc1":cc1,
                        "cc2":cc2,
                        "cc_kind":cc_kind,
                        "bregion":bregion,
                        "twind":twind
                    })
                    # res.append({
                    #     "var_or_other":"var",
                    #     "var":var,
                    #     "cc":cc1,
                    #     "cc_kind":cc_kind,
                    #     "bregion":bregion,
                    #     "twind":twind
                    # })
                    # res.append({
                    #     "var_or_other":"other",
                    #     "var":var_other,
                    #     "cc":cc2,
                    #     "cc_kind":cc_kind,
                    #     "bregion":bregion,
                    #     "twind":twind
                    # })
                    
                    ##### Method 1: include only data-pairs that make different predictions across the var
                    
                    # - normalize so that 0 is same and 1 is most different. This is becuase some, e.g, for
                    # ordinal, have larger range.
                    vec1_norm = vec1/np.max(vec1)
                    vec2_norm = vec2/np.max(vec2)
                    
                    inds_mask = vec1_norm!=vec2_norm
                    print(f"Keeping this many comparisons: {sum(inds_mask)}, out of this many total: {len(inds_mask)}")
                    
                    cc1 = np.corrcoef(vec_dat[inds_mask], vec1[inds_mask])[0,1]
                    cc2 = np.corrcoef(vec_dat[inds_mask], vec2[inds_mask])[0,1]
                    print(var, cc1)
                    print(var_other, cc2)
                    
                    cc_kind = "method1"
                    res.append({
                        "var":var,
                        "var_other":var_other,
                        "cc1":cc1,
                        "cc2":cc2,
                        "cc_kind":cc_kind,
                        "bregion":bregion,
                        "twind":twind
                    })
                    
                    ##### Method 2: Constrain just to cases that are "same" (i.e., dist = 0) in the other var. 
                    # NO: this is not good. this removes all comparisons of var across levels of other_var. This stops 
                    # testing of "abstraction" of var.
                    if False:
                        inds_mask = vec2==1
                        print(f"Keeping this many comparisons: {sum(inds_mask)}, out of this many total: {len(inds_mask)}")
                        
                        cc1 = np.corrcoef(vec_dat[inds_mask], vec1[inds_mask])[0,1]
                        cc2 = np.corrcoef(vec_dat[inds_mask], vec2[inds_mask])[0,1]
                        print(cc1, cc2)
                    
                    ##### Method 3: Constrain just to levels of "othervar" that have at least 2 levels for var. 
                    # - reason: otherwise that level of othervar isn't contributing.
                    from pythonlib.tools.pandastools import conjunction_vars_prune_to_balance, extract_with_levels_of_conjunction_vars
                    
                    # First, get subset of rows that are good.
                    dflab = Clsim.rsa_labels_return_as_df()
                    if PLOT:
                        savedir="/tmp/tmp.png"
                    else:
                        savedir = None
                    dfout1, _ = extract_with_levels_of_conjunction_vars(dflab, var, [var_other], n_min=2, lenient_allow_data_if_has_n_levels=2, plot_counts_heatmap_savedir=savedir, DEBUG=False);
                    dfout2, _ = extract_with_levels_of_conjunction_vars(dflab, var_other, [var], n_min=2, lenient_allow_data_if_has_n_levels=2, plot_counts_heatmap_savedir=savedir, DEBUG=False);
                    inds_rows = [i for i in dfout1["row_index"].tolist() if i in dfout2["row_index"].tolist()]
                    
                    print(len(dflab), " datapts, pruned to --> ", len(inds_rows))
                    
                    vec_dat = Clsim.dataextract_upper_triangular_flattened(True, inds_rows)
                    vec1 = Cltheor1.dataextract_upper_triangular_flattened(True, inds_rows)
                    vec2 = Cltheor2.dataextract_upper_triangular_flattened(True, inds_rows)
                    
                    cc1 = np.corrcoef(vec_dat, vec1)[0,1]
                    cc2 = np.corrcoef(vec_dat, vec2)[0,1]
                    print(var, cc1)
                    print(var_other, cc2)
                    
                    cc_kind = "method3"
                    res.append({
                        "var":var,
                        "var_other":var_other,
                        "cc1":cc1,
                        "cc2":cc2,
                        "cc_kind":cc_kind,
                        "bregion":bregion,
                        "twind":twind
                    })
                    
                    if False:
                        # To get fully square matrix. Not worth it - drops too much data.
                        dfthisout, dfcounts = conjunction_vars_prune_to_balance(dflab, var, var_other, True, prefer_to_drop_which=var);
                    
                    if PLOT:    
                        # Plot correlation
                        fig, ax = plt.subplots()
                        # ax.plot(vec1[inds_mask], vec_dat[inds_mask], "xk", alpha=0.1)
                        ax.plot(vec2[inds_mask], vec_dat[inds_mask], "xk", alpha=0.1)
                        ax.set_xlabel("dist(theor)")
                        ax.set_ylabel("dist(dat)")
                        
                    plt.close("all")

# Plot summary
dfres_cc_pairs = pd.DataFrame(res)

# Heatmap (subplot for each bregion, each showing each pair of variables)
cc_kind = "method1"

if cc_kind=="method1":
    diverge=True
else:
    diverge=False
ncols = 3
W = 4
H = 4
nrows = int(np.ceil(len(list_bregion)/ncols))

# for norm_method in ["all_sub", None, "row_sub", "col_sub"]:
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
for bregion, ax in zip(list_bregion, axes.flatten()):
    print(bregion)
    a = dfres_cc_pairs["bregion"]==bregion
    b = dfres_cc_pairs["cc_kind"]==cc_kind    
    dfthis = dfres_cc_pairs[(a) & (b)]
    convert_to_2d_dataframe(dfthis, "var", "var_other", True, "mean", "cc1", ax=ax, annotate_heatmap=False, diverge=diverge)
    ax.set_title(bregion) 


In [ ]:
# Heatmap (subplot for each bregion, each showing each pair of variables)
cc_kind = "method3"

if cc_kind=="method1":
    diverge=True
else:
    diverge=False
ncols = 3
W = 4
H = 4
nrows = int(np.ceil(len(list_bregion)/ncols))

# for norm_method in ["all_sub", None, "row_sub", "col_sub"]:
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
for bregion, ax in zip(list_bregion, axes.flatten()):
    print(bregion)
    a = dfres_cc_pairs["bregion"]==bregion
    b = dfres_cc_pairs["cc_kind"]==cc_kind    
    dfthis = dfres_cc_pairs[(a) & (b)]
    convert_to_2d_dataframe(dfthis, "var", "var_other", True, "mean", "cc1", ax=ax, annotate_heatmap=False, diverge=diverge)
    ax.set_title(bregion) 


In [ ]:
# Process results from method 1: for each var take mean over all other vars.
from pythonlib.tools.pandastools import aggregGeneral
dfres_cc_pairs_AGG = aggregGeneral(dfres_cc_pairs, ["var", "cc_kind", "bregion", "twind"], ["cc1"], aggmethod=["mean"])
dfres_cc_pairs_AGG

In [ ]:
list_cc_method = dfres_cc_pairs_AGG["cc_kind"].unique().tolist()
cc_kind = "method3"

ncols = 3
W = 4
H = 4
nrows = int(np.ceil(len(list_cc_method)/ncols))


fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
for cc_kind, ax in zip(list_cc_method, axes.flatten()):
    if cc_kind=="method1":
        diverge=True
    else:
        diverge=False
    print(cc_kind)
    # a = dfres_cc_pairs["bregion"]==bregion
    b = dfres_cc_pairs_AGG["cc_kind"]==cc_kind    
    dfthis = dfres_cc_pairs_AGG[(b)]
    convert_to_2d_dataframe(dfthis, "bregion", "var", True, "mean", "cc1", ax=ax, annotate_heatmap=False, diverge=diverge, dosort_colnames=False)
    ax.set_title(cc_kind) 


### Post-hoc plotting things realtied to sim mat

In [ ]:
from neuralmonkey.analyses.state_space_good import _preprocess_rsa_scalar_population

In [ ]:
# correlation between these variables
# TODO: extract values from within _preprocess_rsa_scalar_population to do this.

# %matplotlib inline
# Cltheor = Clsim.rsa_distmat_construct_theoretical("shape_oriented")[0]
# Cltheor.rsa_plot_heatmap()

##### Debug -- plot for specific hand-picked cases

In [ ]:
bregion = "preSMA_a"
twind = (0.1, 0.3)
# twind = (-0.5, -0.3)
which_level = "stroke"

res, PA, Clraw, Clsim = load_single_data(RES, bregion, twind, which_level)



In [ ]:
### 1) PRUNE DATASET
# Restrict just to tasks with 4 prims
print(PA.X.shape)
PAthis = PA.slice_by_labels("trials", "FEAT_num_strokes_task", [3, 4])
print(PAthis.X.shape)
### 2) KEEP SPECIFIC LEVELS
# Keep only specific levels of each of the vars
# var = "stroke_index"
# levels = [0,1,2]
var = "stroke_index_fromlast"
levels = [-4, -3, -2]

PAthis = PAthis.slice_by_labels("trials", var, levels)
print(PA.X.shape)
print(PAthis.X.shape)


In [ ]:
# Generate new Clsim
var = "CTXT_loc_next"
var_other = "shape_oriented"
vars = [var, var_other]
version_distance = "pearson"
subtract_mean_each_level_of_var = vars[1]
# subtract_mean_each_level_of_var = None
_, _, Clraw, Clsim, PAagg = _preprocess_rsa_scalar_population(PAthis, vars, version_distance, subtract_mean_each_level_of_var=subtract_mean_each_level_of_var)


In [ ]:
Cltheor = Clsim.rsa_distmat_construct_theoretical(var, False)[0]
Clraw.rsa_plot_heatmap((0,))
Cltheor.rsa_plot_heatmap((0,))
Clsim.rsa_plot_heatmap((0,))

In [ ]:
fig, axes = plt.subplots(1,2)
print(lev)

ax = axes.flatten()[0]
Clsim._rsa_matindex_plot_bool_mask(ma_same, ax)

ax = axes.flatten()[1]
Clsim._rsa_matindex_plot_bool_mask(ma_diff, ax)

##### DEbugging, constructing theoretical var

In [ ]:
from neuralmonkey.analyses.state_space_good import rsa_convert_PA_to_Cl, popanal_preprocess_scalar
grouping_vars = ["task_kind", "shape_oriented", "stroke_index_fromlast_tskstks"]
PAnorm, PAagg, fig, axes, groupdict = popanal_preprocess_scalar(PA, grouping_vars,
                                                                subtract_mean_each_level_of_var,
                                                                plot_example_chan=False,
                                                                plot_example_split_var=False)

Clraw, Clsim = rsa_convert_PA_to_Cl(PAagg, grouping_vars)


In [ ]:
Clsim.rsa_distmat_construct_theoretical("task_kind", PLOT=True)

In [ ]:
PA.Xlabels["trials"].columns

### [OBSOLETE] Trying to plit automaticlaly into "questions" or "task sets"

Defined as conjucntions of n strokes in task, beh stroke indices, etc.
Problem: Best to determine by hand which params and datapts to include for each "question" above

In [ ]:
### GENERATE DATA FOR EACH TASKSET
# res["list_time_windows"]
DictBregionTwindCPA = res["DictBregionTwindPA"]

grp_task = ["FEAT_num_strokes_task", "stroke_index_0"]
var_taskset = "taskset_nstk_idx0"
min_n_trials_taskset = 20

DictDictBregionTwindCPA = {} # task_set:{(bregion, twind):pa}}
for key, PA in DictBregionTwindCPA.items():
    
    ### 1) PRUNE DATASET
    # Restrict just to tasks with 4 prims
    PA.Xlabels["trials"]["stroke_index_0"] = PA.Xlabels["trials"]["stroke_index"]==0
    PA.Xlabels["trials"] = append_col_with_grp_index(PA.Xlabels["trials"], grp_task, var_taskset, use_strings=False)
    
    # Collect for each task set
    list_task_set = PA.Xlabels["trials"][var_taskset].unique().tolist()
    
    for task_set in list_task_set:
        PAthis = PA.slice_by_labels("trials", var_taskset, [task_set])
        
        print(task_set, PAthis.X.shape)
        if PAthis.X.shape[1] < min_n_trials_taskset:
            # Exlcude
            print("Skipping this taskset, since not enough data: ", task_set, PAthis.X.shape)
            continue
        
        # Collect data
        if task_set in DictDictBregionTwindCPA.keys():
            DictDictBregionTwindCPA[task_set][key] = PAthis
        else:
            DictDictBregionTwindCPA[task_set] = {key:PAthis}
           
        if False:
            ### 2) KEEP SPECIFIC LEVELS
            # Keep only specific levels of each of the vars
            # var = "stroke_index"
            # levels = [0,1,2]
            var = "stroke_index_fromlast"
            levels = [-4, -3, -2]
            
            PAthis = PAthis.slice_by_labels("trials", var, levels)
            print(PA.X.shape)
            print(PAthis.X.shape)
    # DictBregionTwindCPA_NEW[k] = pa

# Summarize 
print(" ========== FINAL SUMMARY (taskset, pa.X.shape)")
for ts, dat in DictDictBregionTwindCPA.items():
    pa = list(dat.values())[0]
    print(ts, " --- ", pa.X.shape)
savedir = f"{SAVEDIR_MULT}/overview_split_by_taskset"
os.makedirs(savedir, exist_ok=True)

# Different effect vars for each dataset...

EffectvarsEachTaskset = {}
list_task_set = list(DictDictBregionTwindCPA.keys())
for ts in list_task_set:
    DictBregionTwindCPA_THIS = DictDictBregionTwindCPA[ts]
    
    # Should be same for each bregion, so just take first.
    pa = list(DictBregionTwindCPA_THIS.values())[0]
    
    # EFFECT_VARS = ["shape_oriented", "gridloc"]
    ####### PRUNE VARS TO SUIT THIS PA
    pa_new, res_check_before, res_check_after, vars_remove, reason_vars_remove = preprocess_prune_pa_enough_data(pa, EFFECT_VARS)
    
    if pa_new is None:
        print(ts)
        print(reason_vars_remove)
        assert False, "these vars just dont work.."
        
    # Exclude all vars that fail because not enough levels, and try again
    vars_exclude = [var for var, reason in reason_vars_remove.items() if reason=="not_enough_levels"]
    EFFECT_VARS_PRUNED = [var for var in EFFECT_VARS if var not in vars_exclude]
    
    pa_new, res_check_before, res_check_after, vars_remove, reason_vars_remove = preprocess_prune_pa_enough_data(pa, EFFECT_VARS_PRUNED)
    if pa_new is None:
        print("Problem, why still pruning a var, after already removed the vars with < 2 levels?")
        print(reason_vars_remove)
        assert False
    else:
        print("Good, vars ready.. got these, for ts:", ts)
        print(EFFECT_VARS_PRUNED)
        
    EffectvarsEachTaskset[ts] = EFFECT_VARS_PRUNED
    
    
    
    

# Check a specific pa
ts = (4, True)
DictBregionTwindCPA_THIS = DictDictBregionTwindCPA[ts]
EFFECT_VARS_PRUNED = EffectvarsEachTaskset[ts]

pa = list(DictBregionTwindCPA_THIS.values())[0]
pa_new, res_check_before, res_check_after, vars_remove, reason_vars_remove = preprocess_prune_pa_enough_data(pa, EFFECT_VARS_PRUNED)

print(vars_remove)
print(reason_vars_remove)
# display(res_check_after["stroke_index_semantic"])

#### First, check that the vars you inputed are actually collectable for the data
# - ie enough data

# Run for each taskset
list_task_set = DictDictBregionTwindCPA.keys()
subtract_mean_each_level_of_var = "IGNORE"
EFFECT_VARS_THIS = res["EFFECT_VARS"]

# Track success
successes = []
for task_set in list_task_set:
    DictBregionTwindCPA_THIS = DictDictBregionTwindCPA[task_set]

#### SCORE EACH DATASET

# Run for each taskset
list_task_set = DictDictBregionTwindCPA.keys()
# EFFECT_VARS = res["EFFECT_VARS"]
subtract_mean_each_level_of_var = "IGNORE"
EFFECT_VARS_THIS = res["EFFECT_VARS"]

# Track success
successes = []
for task_set in list_task_set:
    DictBregionTwindCPA_THIS = DictDictBregionTwindCPA[task_set]
    
    ##
    sdir = f"{savedir}/{task_set}"
    
    PLOT = False
    DFRES_SAMEDIFF, DFRES_THEOR, DictBregionTwindPA, \
        DictBregionTwindClraw, DictBregionTwindClsim, sdir = _pipeline_score_all_pa(DictBregionTwindCPA_THIS, EFFECT_VARS=EFFECT_VARS_THIS,
                           savedir=sdir, PLOT=PLOT, subtract_mean_each_level_of_var=subtract_mean_each_level_of_var)

    resthis = {
        "version_distance":version_distance,
        "which_level":which_level,
        "DFRES_SAMEDIFF":DFRES_SAMEDIFF, 
        "DFRES_THEOR":DFRES_THEOR, 
        "DictBregionTwindPA":DictBregionTwindPA, 
        "DictBregionTwindClraw":DictBregionTwindClraw,
        "DictBregionTwindClsim":DictBregionTwindClsim,
        "EFFECT_VARS":EFFECT_VARS_THIS,
        "list_time_windows":res["list_time_windows"],
        "SAVEDIR":sdir,
        "subtract_mean_each_level_of_var":subtract_mean_each_level_of_var,
    }
    
    # Save results temporarily
    # Around 800 MB
    import pickle
    path = f"{sdir}/resthis.pkl"
    with open(path, "wb") as f:
        pickle.dump(resthis, f)
        print("saved res to: ", path)
        
    for k, clraw in DictBregionTwindClraw.items():
        successes.append({
            "task_set":task_set,
            "bregion":k[0],
            "twind":k[1],
            "success":clraw is not None
        })
        
dfsucc = pd.DataFrame(successes)
dfsucc[dfsucc["success"]==False]

### Try using all trials

In [ ]:
animal = "Pancho"
DATE = 230623
list_which_level=["stroke"]
RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, "pearson", list_which_level)


In [ ]:
bregion = "dlPFC_a"
twind = (0.1, 0.3)
DictBregionTwindPA = RES[0]["DictBregionTwindPA"]

PA = DictBregionTwindPA[(bregion, twind)]

In [ ]:
from neuralmonkey.analyses.rsa import _preprocess_rsa_scalar_population
vars = ["shape_oriented", "gridloc"]
DO_AGG_TRIALS = False
# subtract_mean_each_level_of_var = "gridloc"
subtract_mean_each_level_of_var = "shape_oriented"
subtract_mean_each_level_of_var = None
dfres_same_diff, dfres_theor, Clraw, Clsim, PAagg = _preprocess_rsa_scalar_population(PA, vars, 
                                                                                      subtract_mean_each_level_of_var=subtract_mean_each_level_of_var,
                                                                                      PLOT=False, sdir="/tmp", PLOT_THEORETICAL_SIMMATS=False, DO_AGG_TRIALS=DO_AGG_TRIALS)



In [ ]:
Clraw.rsa_plot_heatmap(sort_order=(0,1))

In [ ]:
Clsim.rsa_plot_heatmap(sort_order=(0,1));


In [ ]:
Cltheor = Clsim.rsa_distmat_construct_theoretical("gridloc")[0]
Cltheor.rsa_plot_heatmap((0,1))